## Explore the neighborhoods in the city of Toronto

<a id='item1'></a>

# Import libraries

In [102]:
import numpy as np
import pandas as pd
import requests

!conda install -c conda-forge beautifulsoup4 --yes
import bs4
from bs4 import BeautifulSoup

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.8.3
  latest version: 4.8.4

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



# Get Data from URL 

In [104]:
request_data = requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M').text
soup = BeautifulSoup(request_data, "html.parser")

# Find the data in the table cells and construct a dataframe

In [105]:
table=soup.find('table')
#dataframe will consist of three columns: PostalCode, Borough, and Neighborhood
df = pd.DataFrame(columns = ['PostalCode','Borough','Neighborhoods'])
df.shape

# Search all the postcode, borough, neighborhood 
for trcelldata in table.find_all('tr'):
    trdata=[]
    for tdcelldata in trcelldata.find_all('td'):
        trdata.append(tdcelldata.text.strip())
    if len(trdata)==3:
        df.loc[len(df)] = trdata

# Perform the data cleanup and grooming required in lab

In [106]:
df.dropna()
# drop Borough not assigned 
df.drop(df[ df['Borough'] == 'Not assigned' ].index, inplace = True) 

# assign Borough to Neighborhood if latter not assigned
df.Neighborhoods[df.Neighborhoods == 'Not assigned'] = df.Borough

# Create a second frame grouping by postal code with neighborhoods transformed by , concatennation
# In the dataset this is already done so no need to do this, but doing it because it is part of assignment 
df1=df.groupby('PostalCode')['Neighborhoods'].apply(lambda x: "%s" % ', '.join(x))
# Merge , drop the extra column not comma separated and rename to correct name
df2 = pd.merge(df, df1, on='PostalCode').drop_duplicates().drop(['Neighborhoods_x'],axis=1).rename(columns={'Neighborhoods_y':'Neighborhoods'})

df2.shape


(103, 3)